# EV Charger Data Cleanup for the Energy Office

June Chee, Transportation Energy Analyst with the Hawaii State Energy Office requested some assistance cleaning up their EV Charging Data. The data consists of three files, all available in this folder

1. **Hawaii EV App_Charging Stations List**: This is the charging station app that the Hawaii State Energy Office manages. Ideally the new data set would help to clean up and repopulate the information for this app. You can see how the app looks here: http://energy.hawaii.gov/testbeds-initiatives/ev-ready-program/ev-stations-hawaii-mobile-app

- **AFDC Hawaii Charging Stations List**: This is the list you on the federal Alternative Fuels Data Center. You can access the station locater here: https://www.afdc.energy.gov/fuels/electricity_locations.html#/analyze?region=HI

- **ChargePoint_Hawaii EV Charging Stations**: this is a data set from one of the major EV charging station vendors in Hawaii who have recently made updates and new installments to charging stations.

In [41]:
import pandas as pd
import numpy as np

In [42]:
Chargepoint = pd.read_csv("Chargepoint.csv")
EVApp =pd.read_csv("EVApp.csv")

## EV App Dataset

This is the target dataset. It has **257 Locations** and some specific geography including address, lattitude and longitude. The stations have ids but presumably that's within this system and does not have any other matching features in other datasets. Filtering out duplicates may be a matter of matching latitude and logitude... perhaps within a few feet? Or Address matches?

In [43]:
EVApp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 23 columns):
Unnamed: 0              257 non-null int64
Station ID              257 non-null int64
Station name            257 non-null object
Location                257 non-null object
Island                  257 non-null object
Address                 257 non-null object
City                    257 non-null object
State                   257 non-null object
Zip Code                257 non-null int64
Latitude                257 non-null object
Longitude               257 non-null object
Contact Name            118 non-null object
Charge Fee              255 non-null object
Park Fee                190 non-null object
Parking Lot             99 non-null object
Hours                   255 non-null object
Ports                   257 non-null int64
Restrictions            165 non-null object
Number of Stations      257 non-null int64
Charge Fees             256 non-null object
Manufacturers        

## Chargepoint Data
This is the dataset solely for chargepoint, probably the biggest individual provider? It has Addresses, but not latitudes and Longitudes. It might be possible to run these through a converter, or just to use the addresses. 

In [44]:
Chargepoint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 12 columns):
Unnamed: 0                 148 non-null int64
Station Name               148 non-null object
Org Name                   148 non-null object
Address 1                  148 non-null object
City                       148 non-null object
State                      148 non-null object
Postal Code                148 non-null int64
County                     140 non-null object
Country                    148 non-null object
No. of Ports               148 non-null int64
Station Activation Type    148 non-null object
Customer Category          148 non-null object
dtypes: int64(3), object(9)
memory usage: 14.0+ KB


## Step 1: In EVApp, filter out all chargers that are not Chargepoint

In [45]:
EVApp_CPOnly = EVApp.loc[EVApp['Manufacturers']=='ChargePoint']

## Step 1: Prefix column names to identify data set source
this is needed because we will eventually create a dataset that includes both evapp and chargepoint datasets.  
we will want to identify columns that came from EVApp versus columns that came from Chargepoint

In [46]:
#prefix all column names with source
EVApp.rename(columns = lambda x: 'EV_'+x, inplace=True)
Chargepoint.rename(columns = lambda x: 'CH_'+x, inplace=True)

## Step 2: Prepare addresses to use as an identifier between the datasets.
Try using address.  We will need to make address conform to same format.  

Format:  
1.  All upper case letters
2.  No okina  
3.  Spell out all abbreviations.
4.  Remove periods


In each data set, create a new column called 'Common Address'.  

Right off the bat you can see that some addresses live in all three, but also that some contain duplicate listings in a single datasource. 

For example:  
100 Ho'okele St	EVApp  
100 Ho'okele St	Chargepoint  
100 Ho'okele St	Chargepoint  

In [47]:
df_streetabrevs = pd.read_csv(r'Street Abbreviations.csv') #https://mbiz.bf.umich.edu/mailsvcs/streetsuffixes.htm
df_streetabrevs = df_streetabrevs.append(pd.DataFrame([['North','N'],['South','S'],['EAST','E'],['WEST','W']], columns=['Full','Abbreviation']), ignore_index = True)

def _makeCommonAddress(Address):
#     take in an address string, return a common address string
    CommonAddress = Address #copy address string
    #do some initial transformations:
    CommonAddress = CommonAddress.upper()    
    CommonAddress = CommonAddress.replace('','') #remove okina
    CommonAddress = CommonAddress.replace('.','')
    CommonAddress = CommonAddress.replace(',','')
    CommonAddress = CommonAddress.replace('-','')  
    CommonAddress = CommonAddress.replace('/','')
    CommonAddress = CommonAddress.replace('\\','')
    CommonAddress = CommonAddress.replace("'",'') #remove okina    
    CommonAddress = CommonAddress.replace("’",'') #remove okina
    #before doing abbreviations transform, change all street conversions to upper case
#     CommonAddress = CommonAddress.upper()  
     #replace street abbreviations w/ full:
    ls_parts = CommonAddress.split(' ')
    def replace_part(part):
        val = df_streetabrevs.loc[df_streetabrevs['Abbreviation']==part,'Full']
        if len(val) > 0: return val.iloc[0]
        else: return part
    ls_parts_rev = [replace_part(part) for part in ls_parts]
    CommonAddress = ''.join(ls_parts_rev)
    #change all street conversions to upper case
    CommonAddress = CommonAddress.upper()    
    return CommonAddress

In [48]:
#here we make new dataframes using old ones that includes a common address column
EVApp['Cmn_Address'] = EVApp.apply(lambda row: _makeCommonAddress(row['EV_Address']), axis=1)
Chargepoint['Cmn_Address'] = Chargepoint.apply(lambda row: _makeCommonAddress(row['CH_Address 1']), axis=1)

## Step 3: remove duplicate addresses 
In EVApp & Chargepoint (separately), select only unique rows using Cmn_Address column as unique key identifier  
**FUTURE WORK NOTE: we need to consider implications of ignoring data that we lose when removing duplicate address rows based on address.  

In [49]:
EVApp = EVApp.groupby(['Cmn_Address'], as_index=False).first() #https://stackoverflow.com/questions/12322779/pandas-unique-dataframe
Chargepoint = Chargepoint.groupby(['Cmn_Address'], as_index=False).first()

EVApp.to_csv('EVApp_unique_addresses.csv', encoding='utf-8')
Chargepoint.to_csv('Chargepoint_unique_addresses.csv', encoding='utf-8')

## Step 4: Revise EVApp to only include chargepoint manufacturers 

In [50]:
EVApp = EVApp.loc[EVApp['EV_Manufacturers'] == 'ChargePoint']
EVApp.to_csv('EVApp_CP_unique_addresses.csv', encoding='utf-8')


## Step 4: Merge EVApp & Chargepoint into a master data set

In [37]:
MasterData = pd.merge(EVApp, Chargepoint, on = 'Cmn_Address', how = 'outer')

## Step 5: Make Only_In_EV.csv
use MasterData data set to create a file that contains records existing in EVApp but not chargepoint

In [38]:
Only_In_EV = MasterData.loc[MasterData['EV_Address'].notna() & MasterData['CH_Address 1'].isna()]
Only_In_EV.to_csv('Only_In_EV.csv', encoding='utf-8')
display(Only_In_EV)

,Cmn_Address,EV_Unnamed: 0,EV_Station ID,EV_Station name,EV_Location,EV_Island,EV_Address,EV_City,EV_State,EV_Zip Code,...,CH_Org Name,CH_Address 1,CH_City,CH_State,CH_Postal Code,CH_County,CH_Country,CH_No. of Ports,CH_Station Activation Type,CH_Customer Category
0,1001KAMOKILABOULEVARDKAPOLEIBUILDING,257.0,905.0,Campbell Square,P1 across from elevators,Oahu,"1001 Kamokila Blvd, Kapolei Building",Kapolei,HI,96707.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1170NUUANU,94.0,489.0,Smith-Beretania Parking Structure,Entrance on Beretania Street between Nuuanu Av...,Oahu,1170 Nuuanu,Honolulu,HI,96817.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1380LUSITANASTREET,75.0,447.0,Queen’s Physician Office Building,1 Charger at POB 1 lot on Lusitaina St. Main e...,Oahu,1380 Lusitana street,Honolulu,HI,96813.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2345KUHIOAVENUE,242.0,869.0,International Market Place,.,Oahu,2345 Kuhio Ave,Honolulu,HI,96815.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,3221WAIALAEAVENUE,52.0,395.0,Kaimuki Shopping Center,The charging station is in the parking lot.,Oahu,3221 Waialae Ave,Honolulu,HI,96816.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,410KOLOASTREET,143.0,641.0,BMW of Maui,The charger is located in the outside parking lot,Maui,410 Koloa Street,Kahului,HI,96732.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,4450KAPOLEIPARKWAYSTE100,159.0,673.0,Target-Kapolei,Charger located in front of exit doors,Oahu,4450 Kapolei Pkwy Ste 100,Kapolei,HI,96707.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,724ILANIWAI,223.0,819.0,Green Energy Outlet,Green spot for EV Charging in front of lot. Ot...,Oahu,724 Ilaniwai,Honolulu,HI,96813.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 6: Make Only_In_CH.csv
use MasterData data set to create a file that contains records existing in chargepoint but not EVApp

In [39]:
Only_In_CH = MasterData.loc[MasterData['CH_Address 1'].notna() & MasterData['EV_Address'].isna()]
Only_In_CH.to_csv('Only_In_CH.csv', encoding='utf-8')
display(Only_In_CH.head(10))

,Cmn_Address,EV_Unnamed: 0,EV_Station ID,EV_Station name,EV_Location,EV_Island,EV_Address,EV_City,EV_State,EV_Zip Code,...,CH_Org Name,CH_Address 1,CH_City,CH_State,CH_Postal Code,CH_County,CH_Country,CH_No. of Ports,CH_Station Activation Type,CH_Customer Category
29,1000ULUOHIASTREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,City & County of Honolulu,1000 Ulu'Ohi'a St,Kapolei,Hawaii,96707.0,Honolulu County,United States,1.0,Public,Municipal
30,1001KAMOKILABOULEVARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,James Campbell Company LLC co MMI Realty Servi...,1001 Kamokila Blvd,Kapolei,Hawaii,96707.0,Honolulu County,United States,2.0,Public,Workplace
31,101MALUNIUAVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Highmark Capital LLC,101 Maluniu Ave,Kailua,Hawaii,96734.0,Honolulu County,United States,2.0,Public,Multifamily Commercial
32,111911234THAVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,CBI Inc. Hawaii,1119-1123 4th Ave,Honolulu,Hawaii,96816.0,Honolulu County,United States,2.0,Public,Retail
33,11511185WAIANUENUEAVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Hawaii Health Systems Hilo Medical Center,1151-1185 Waianuenue Ave,Hilo,Hawaii,96720.0,Hawaii County,United States,2.0,Public,Healthcare
34,1170NUUANUAVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,City & County of Honolulu,1170 Nuuanu Ave,Honolulu,Hawaii,96817.0,Honolulu County,United States,1.0,Public,Municipal
35,1189WAIMANUSTREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Waihonua at Kewalo,1189 Waimanu St,Honolulu,Hawaii,96814.0,Honolulu County,United States,1.0,Public,Workplace
36,1234SOUTHBERETANIASTREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"MMI Realty Services, Inc.",1234 S. Beretania St,Honolulu,Hawaii,96814.0,Honolulu County,United States,2.0,Public,Workplace
37,1260KUALASTREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Hickam Federal Credit Union,1260 Kuala St,Pearl City,Hawaii,96782.0,Honolulu County,United States,1.0,Public,Workplace
38,1288ALAMOANABOULEVARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Hokua Honolulu,1288 Ala Moana Blvd,Honolulu,Hawaii,96814.0,Honolulu County,United States,2.0,Public,Multifamily Commercial


## Step 7: Make In_EV_CH.csv
use MasterData data set to create a file that contains records existing in chargepoint but not EVApp

In [40]:
In_EV_CH = MasterData.loc[MasterData['EV_Address'].notna() & MasterData['CH_Address 1'].notna()]
In_EV_CH.to_csv('In_EV_CH.csv', encoding='utf-8')
display(In_EV_CH)

,Cmn_Address,EV_Unnamed: 0,EV_Station ID,EV_Station name,EV_Location,EV_Island,EV_Address,EV_City,EV_State,EV_Zip Code,...,CH_Org Name,CH_Address 1,CH_City,CH_State,CH_Postal Code,CH_County,CH_Country,CH_No. of Ports,CH_Station Activation Type,CH_Customer Category
1,100HOOKELESTREET,198.0,765.0,Target Corp Maui - Kahului,Parking lot,Maui,100 Ho'okele St,Kahului,HI,96732.0,...,Target Corporation,100 Ho'okele St,Kahului,Hawaii,96732.0,Maui County,United States,2.0,Public,Retail
2,100NOHEAKAIDRIVE,23.0,315.0,Marriott Maui Ocean Club,The charging station is located in the Valet s...,Maui,100 Nohea Kai Drive,Lahaina,HI,96761.0,...,Marriott MOC,100 Nohea Kai Dr,Lahaina,Hawaii,96761.0,Maui County,United States,1.0,Public,Hospitality
3,1031NUUANUAVENUE,95.0,491.0,Chinatown Gateway Plaza Parking Structure,The entrance to the parking structure is locat...,Oahu,1031 Nuuanu Ave,Honolulu,HI,96817.0,...,City & County of Honolulu,1031 Nuuanu Ave,Honolulu,Hawaii,96817.0,Honolulu County,United States,1.0,Public,Retail
6,151KAPAHULUAVENUE,92.0,485.0,Honolulu Zoo Parking Lot (Currently DOWN),Entrance to parking lot is on Kapahulu. Follo...,Oahu,151 Kapahulu Avenue,Honolulu,HI,96815.0,...,City & County of Honolulu,151 Kapahulu Ave,Honolulu,Hawaii,96815.0,Honolulu County,United States,1.0,Public,Hospitality
7,155NORTHBERETANIASTREET,96.0,493.0,Hale Pauahi Parking Structure,The entrance is on Beretania Street. Follow b...,Oahu,155 N Beretania St.,Honolulu,HI,96817.0,...,City & County of Honolulu,155 N Beretania St,Honolulu,Hawaii,96817.0,Honolulu County,United States,1.0,Public,Multifamily Commercial
8,200NOHEAKAIDRIVE,19.0,307.0,Hyatt Regency Maui,"In the Lahaina parking area, take a left turn ...",Maui,200 Nohea Kai Drive,Lahaina,HI,96761.0,...,Hyatt Regency Lahaina Maui,200 Nohea Kai Dr,Lahaina,Hawaii,96761.0,Maui County,United States,2.0,Public,Hospitality
11,345HAHANISTREET,184.0,735.0,Target- Kailua,Target,Oahu,345 Hahani St,Kailua,HI,96734.0,...,Target Corporation,345 Hahani St,Kailua,Hawaii,96734.0,Honolulu County,United States,2.0,Public,Retail
12,391MAKAALASTREET,183.0,733.0,Target- Hilo,"In front of store, near ADA spot",Hawaii,391 Makaala Street,Hilo,HI,96720.0,...,Target Corporation,391 Makaala St,Hilo,Hawaii,96720.0,Hawaii County,United States,2.0,Public,Retail
13,3970KAANASTREET,195.0,759.0,5th Circuit Court,Southwest corner of the building; Gated,Kauai,3970 Kaana St,Lihue,HI,96766.0,...,5th Circuit Court,3970 Kaana St,L?hu?e,Hawaii,96766.0,Kauai County,United States,1.0,Public,Workplace
15,4380LAWEHANASTREET,158.0,671.0,Target- Honolulu,Middle row of parking lot - ground floor near ...,Oahu,4380 Lawehana St,Honolulu,HI,96818.0,...,Target Corporation,4380 Lawehana St,Honolulu,Hawaii,96818.0,Honolulu County,United States,2.0,Public,Retail
